In [1]:
!pip install -U sentence-transformers
!pip install pandas
!pip install accelerate

In [ ]:
from sentence_transformers import SentenceTransformer
import pandas as pd
import sklearn.metrics.pairwise as pairwise

filepath = './articles_simplified_4.jsonl'
articles = pd.read_json(path_or_buf=filepath, lines=True)
texts = articles['text']
simplifieds = articles['simplified']
simplifieds2 = articles['simplified2']
simplifieds3 = articles['simplified3']
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

csv_str = "article_id, sim1, sim2, sim3\n"

for i in range(articles.shape[0]):
  sentences = [texts[i], simplifieds[i], simplifieds2[i], simplifieds3[i]]
  embeddings = model.encode(sentences)
  similarities = pairwise.cosine_similarity(embeddings)
  sim1 = similarities[0][1]
  sim2 = similarities[0][2]
  sim3 = similarities[0][3]
  csv_str += f"{i}, {sim1}, {sim2}, {sim3}\n"

with open('similarities.csv', 'w') as f:
  f.write(csv_str)
  f.close()



[[1.0000001  0.94275415 0.81226915 0.94649434]
 [0.94275415 1.         0.77951986 0.90231025]
 [0.81226915 0.77951986 1.         0.805584  ]
 [0.94649434 0.90231025 0.805584   1.        ]]
[[0.9999999  0.8691579  0.8766748  0.9378818 ]
 [0.8691579  1.0000002  0.8663049  0.89241874]
 [0.8766748  0.8663049  0.9999999  0.8962662 ]
 [0.9378818  0.89241874 0.8962662  1.0000002 ]]
[[1.0000001  0.82550514 0.79634863 0.93291104]
 [0.82550514 1.         0.736037   0.79945916]
 [0.79634863 0.736037   1.         0.8309454 ]
 [0.93291104 0.79945916 0.8309454  1.        ]]
[[1.        0.6989082 0.7560197 0.8751534]
 [0.6989082 1.0000001 0.8665192 0.8069662]
 [0.7560197 0.8665192 1.        0.8358339]
 [0.8751534 0.8069662 0.8358339 1.       ]]
[[1.         0.8396716  0.8983436  0.91467464]
 [0.8396716  0.99999994 0.8800962  0.82478034]
 [0.8983436  0.8800962  1.         0.8571048 ]
 [0.91467464 0.82478034 0.8571048  1.0000001 ]]
[[1.0000002  0.71975493 0.90195644 0.9019327 ]
 [0.71975493 1.0000002  